In [7]:
import requests
from datetime import datetime as dt
from datetime import timedelta as td
import pandas as pd
import time

In [8]:
# Ancilliary function that returns each day between a date range 
def daterange(start_date, end_date):
    
    '''This function extracts by performing get calls on the polygon grouped daily bars API endpoint.
        start_date: datetime
        end_date: datetime 
        return: list of datetime objects

    '''
    print('Total days in daterange',int ((end_date - start_date).days))
    for n in range(int ((end_date - start_date).days)):
        yield start_date + td(n)


In [9]:
## Extraction Function
def extract(start_date,end_date):
    '''This function extracts by performing get calls on the polygon grouped daily bars API endpoint.
        start_date: str
        end_date: str 
        return: pd.DataFrame containing ticker, date and stock details 

    '''

    print("Extracting")
    # Type conversions for parsing string as argument to "get" 
    s_date = dt.strptime(start_date,"%Y-%m-%d")
    e_date = dt.strptime(end_date,"%Y-%m-%d")
    
    # Temporary Dataframe that stores daily results
    result_df = pd.DataFrame()
    call_limit = 0
    # Iterate for each day and store results in the temporary dataframe
    for single_date in daterange(s_date, e_date):
        
        current_day_str = single_date.strftime("%Y-%m-%d")
        result = requests.get("https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/"+current_day_str+"?adjusted=true&include_otc=true&apiKey=pgii3xgdAKOYGUUFQHqszfF1kFhH0Hbe")
        if result.status_code == 200 and result.text is not None and result.json()['resultsCount']!=0:
            print("Appending for date: ",current_day_str)
            result_df = result_df.append(pd.DataFrame(result.json()['results']))
            call_limit+=1
        
        if call_limit==4:
            print("API Call Limit Reached. Waiting for 1 minute")
            time.sleep(60)
            print("Active again")
            call_limit = 0
                
    return result_df


In [12]:
# Transform and Load Function
def transform_load(df):
    '''
       This function performs the necessary transformations required on the extracted 
       Dataframe and stores the results in a pickle file(.pkl) 
       df: pd.DataFrame
       return: 

    '''
    
    print("Performing Necessary Transformations")
    column_dict = {'T':'Ticker','v':'Volume','vw':'VWAP','o':'','c':'','h':'','l':'','t':'Timestamp','n':'TransCount'}
    df = df.rename(columns = column_dict)

    df['Timestamp'] = df['Timestamp'].apply(lambda x:(dt.utcfromtimestamp(x/1000).strftime("%Y-%m-%d")))
    df.to_pickle("C:/Users/abhij/Git Repos/DeepLearningLabs/RNNs_in_Action/training.pkl")
     
    

In [13]:
transform_load(extract("2021-09-07","2021-09-14"))

Extracting
Total days in daterange 7
Appending for date:  2021-09-07


C:\Users\abhij\AppData\Local\Temp\ipykernel_11528\3458172334.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(pd.DataFrame(result.json()['results']))


Appending for date:  2021-09-08


C:\Users\abhij\AppData\Local\Temp\ipykernel_11528\3458172334.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(pd.DataFrame(result.json()['results']))


Appending for date:  2021-09-09


C:\Users\abhij\AppData\Local\Temp\ipykernel_11528\3458172334.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(pd.DataFrame(result.json()['results']))


Appending for date:  2021-09-10
API Call Limit Reached. Waiting for 1 minute


C:\Users\abhij\AppData\Local\Temp\ipykernel_11528\3458172334.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(pd.DataFrame(result.json()['results']))


Active again
Appending for date:  2021-09-13
Performing Necessary Transformations


C:\Users\abhij\AppData\Local\Temp\ipykernel_11528\3458172334.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(pd.DataFrame(result.json()['results']))
